In [1]:
# !pip freeze > requirements.txt
# !pip install -r requirements.txt

In [13]:
import os
import json
from datetime import datetime
import pytz
from collections import Counter
import random
random.seed(42)

import numpy as np
import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import (Dataset,
                              DataLoader,
                              random_split,
                              WeightedRandomSampler)
from torch.utils.tensorboard import SummaryWriter
from torchvision import models, transforms
from torchsummary import summary
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import (confusion_matrix,
                             accuracy_score,
                             precision_score,
                             recall_score,
                             f1_score,
                             roc_auc_score)

import warnings
warnings.filterwarnings("ignore")

# Preprocessing

In [3]:
class PathLabelProcessor:
    def __init__(self, 
                 base_path,
                 folder_name,
                 pet_type,
                 lesion,
                 devices,
                 symptom):
        self.base_path = base_path
        self.folder_name = folder_name
        self.pet_type = pet_type
        self.lesion = lesion
        self.devices = devices
        self.symptom = symptom
        
        self.label_images()
        
    def find_folders_by_name(self):
        matching_folders = []

        for root, dirs, files in os.walk(self.base_path):
            for dir_name in dirs:
                if self.folder_name in dir_name:
                    folder_path = os.path.join(root, dir_name)
                    matching_folders.append(folder_path)

        for folder_path in matching_folders:
            print(folder_path)
            
        return matching_folders

    def find_image_json_pairs(self):
        image_paths = []
        json_paths = []

        for folder_path in self.find_folders_by_name():
            for root, dirs, files in os.walk(folder_path):
                for image_file in filter(lambda x: x.lower().endswith(('jpg', 'png')), files):
                    image_path = os.path.join(root, image_file)
                    json_file = f"{os.path.splitext(image_path)[0]}.json"
                    if os.path.isfile(json_file):
                        image_paths.append(image_path)
                        json_paths.append(json_file)

        print(f'Total images: {len(image_paths)}, Total JSON files: {len(json_paths)}')
        
        return image_paths, json_paths

    def label_images(self):
        self.labeled_image_paths = []

        for image_path, json_path in zip(*self.find_image_json_pairs()):
            with open(json_path) as f:
                data = json.load(f)
                
            is_symptomatic = data['label']['label_disease_lv_3'] in self.symptom and data['label']['label_disease_nm'] == self.lesion

            if data['images']['meta']['device'] in self.devices:
                if self.pet_type in os.path.dirname(image_path).lower():
                    self.labeled_image_paths.append((image_path, 0 if is_symptomatic else 1))
                else:
                    self.labeled_image_paths.append((image_path, 1))
        
        self.symptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 0))
        self.asymptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 1))

        print(f'Total cases: {len(self.labeled_image_paths)}')
        print(f'Number of symptomatic cases: {self.symptomatic_count}, Number of asymptomatic cases: {self.asymptomatic_count}')

In [4]:
%%time
base_path = 'eye/Train'
folder_name = '일반'
'''
['유']
dog: 안검염, 안검종양, 안검내반증, 유루증, 색소침착성각막염, 핵경화, 결막염
cat: 안검염, 결막염, 각막부골편, 비궤양성각막염, 각막궤양
['상', '하']
dog: 궤양성각막질환, 비궤양성각막질환
['초기', '비성숙', '성숙']
dog: 백내장
'''
pet_type = '개'
lesion = '안검종양'
devices = ['스마트폰', '일반카메라']
symptom = ['유']

processor = PathLabelProcessor(base_path=base_path,
                               folder_name=folder_name,
                               pet_type=pet_type,
                               lesion=lesion,
                               devices=devices,
                               symptom=symptom)

labeled_image_paths = processor.labeled_image_paths

eye/Train/고양이/안구/일반
eye/Train/개/안구/일반
Total images: 199816, Total JSON files: 199816
Total cases: 98646
Number of symptomatic cases: 52, Number of asymptomatic cases: 98594
CPU times: user 5.87 s, sys: 1.61 s, total: 7.47 s
Wall time: 7.48 s


In [11]:
class ImageDataset(Dataset):
    def __init__(self, labeled_image_paths, transform, augmentation=None):
        self.labeled_image_paths = labeled_image_paths
        self.transform = transform
        self.augmentation = augmentation

        self.labels = [label for _, label in labeled_image_paths]
        self.minority_class = min(Counter(self.labels).items(), key=lambda x: x[1])[0]

    def __len__(self):
        return len(self.labeled_image_paths)

    def __getitem__(self, idx):
        image_path, label = self.labeled_image_paths[idx]

        image = Image.open(image_path)

        transform = [
            self.augmentation
            if self.augmentation and label == self.minority_class
            else self.transform,
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ]

        image = transforms.Compose(transform)(image)

        return image, label

In [12]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240))])
augmentation = transforms.Compose([transforms.RandomHorizontalFlip(),
                                   transforms.RandomVerticalFlip(),
                                   transforms.RandomRotation(degrees=10),
                                   transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)])

dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform,
                       augmentation=augmentation)

CPU times: user 30.4 ms, sys: 0 ns, total: 30.4 ms
Wall time: 29.9 ms


In [ ]:
class DataLoaderMaker:
    def __init__(self,
                 dataset,
                 batch_size,
                 train_ratio=None,
                 num_workers=None,
                 class_weights=None):
        self.dataset = dataset
        self.train_ratio = train_ratio
        self.batch_size = batch_size
        self.num_workers = num_workers

        if train_ratio:
            self.split_and_make_dataloader()
        else:
            self.dataloader = self.make_dataloader(self.dataset)

    def make_dataloader(self, dataset, sampler=None, shuffle=False):
        if sampler is not None:
            dataloader = DataLoader(dataset=dataset,
                                    batch_size=self.batch_size,
                                    sampler=sampler,
                                    num_workers=self.num_workers,
                                    pin_memory=True)
        else:
            dataloader = DataLoader(dataset=dataset,
                                    batch_size=self.batch_size,
                                    shuffle=shuffle,
                                    num_workers=self.num_workers,
                                    pin_memory=True)
        
        self.inspect_data(dataloader)
        
        return dataloader

    def split_and_make_dataloader(self):
        train_size = int(len(self.dataset) * self.train_ratio)
        test_size = len(self.dataset) - train_size
        train_dataset, test_dataset = random_split(self.dataset, [train_size, test_size])

        sampler = WeightedRandomSampler(weights, num_samples=len(train_dataset), replacement=True)

        self.train_loader = self.make_dataloader(train_dataset, sampler=sampler, shuffle=False)
        self.test_loader = self.make_dataloader(test_dataset, shuffle=True)

    def inspect_data(self, dataloader):
        print("Inspecting Data...")

        class_counts = {}
        for _, labels in dataloader:
            labels = labels.type(torch.int)
            for label in labels.tolist():
                class_counts[label] = class_counts.get(label, 0) + 1

        print("- Class Counts:")
        for class_label, count in class_counts.items():
            print(f"  Class {class_label}: {count} samples")

In [ ]:
%%time
batch_size = 96
num_workers = os.cpu_count()
train_ratio = 0.8
weight_class_0 = 1.0 / processor.symptomatic_count
weight_class_1 = 1.0 / processor.asymptomatic_count
class_weights = torch.tensor([weight_class_0, weight_class_1])

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              train_ratio=train_ratio,
                              class_weights=class_weights)

Inspecting Data...
- Class Counts:
  Class 1: 78874 samples
  Class 0: 42 samples
Inspecting Data...
- Class Counts:
  Class 1: 19720 samples
  Class 0: 10 samples
CPU times: user 22.2 s, sys: 20.3 s, total: 42.5 s
Wall time: 50.4 s


# Modeling

In [7]:
class ModelTrainer:
    def __init__(self, 
                 model,
                 device,
                 train_dataloader,
                 valid_dataloader,
                 loss_fn,
                 optimizer,
                 scheduler,
                 pet_type,
                 lesion):
        self.device = device
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.loss_fn = loss_fn.to(device)
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_f1_score = 0.0
        korea = pytz.timezone('Asia/Seoul')
        now = datetime.now(korea)
        start_time = now.strftime('%Y%m%d-%H%M%S')
        self.name = f'{start_time}_{pet_type}_{lesion}.pth'
        self.writer = SummaryWriter(log_dir=f'runs/{self.name}')

    def calculate_f1_score(self, predicted, labels):
        return f1_score(labels, predicted, average='binary')

    def calculate_auc_roc(self, predicted, labels):
        return roc_auc_score(labels, predicted)

    def train_one_epoch(self, epoch, num_epochs):
        self.model.train()
        total_loss = 0.0
        correct = 0
        total = 0
    
        for step, (inputs, labels) in enumerate(tqdm(self.train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')):
            inputs, labels = inputs.to(self.device), labels.to(self.device)
    
            outputs = self.model(inputs)
            
            loss = self.loss_fn(outputs, labels)
            total_loss += loss.item()
    
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
            loss.backward()
    
            self.optimizer.step()
            self.optimizer.zero_grad()
    
        self.scheduler.step()
        self.writer.add_scalar('LearningRate',
                               self.scheduler.get_last_lr()[0],
                               epoch)
    
        avg_loss = total_loss / len(self.train_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/train', avg_loss, epoch)        
        self.writer.add_scalar('Accuracy/train', accuracy, epoch)

    def eval_one_epoch(self, epoch):
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in self.valid_dataloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
        
                outputs = self.model(inputs)
        
                loss = self.loss_fn(outputs, labels)
                total_loss += loss.item()
        
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_loss = total_loss / len(self.valid_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/valid', avg_loss, epoch)
        self.writer.add_scalar('Accuracy/valid', accuracy, epoch)
        
        current_f1_score = self.calculate_f1_score(np.array(all_predicted),
                                                   np.array(all_labels))
        current_auc_roc = self.calculate_auc_roc(np.array(all_predicted),
                                                 np.array(all_labels))
        
        self.writer.add_scalar('F1 Score/valid', current_f1_score, epoch)
        self.writer.add_scalar('AUC-ROC/valid', current_auc_roc, epoch)
        
        if current_f1_score > self.best_f1_score:
            self.best_f1_score = current_f1_score
            torch.save(self.model, self.name)

    def train(self, num_epochs):
        for epoch in range(num_epochs):
            self.train_one_epoch(epoch, num_epochs)
            self.eval_one_epoch(epoch)
            
        self.writer.close()

In [8]:
model = models.efficientnet_b1()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_ftrs = model.classifier[1].in_features
num_classes = 2
model.classifier[1] = nn.Linear(num_ftrs, num_classes)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(model.parameters(), weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

trainer = ModelTrainer(model=model,
                       device=device,
                       train_dataloader=data_loader.train_loader,
                       valid_dataloader=data_loader.test_loader,
                       loss_fn=loss_fn,
                       optimizer=optimizer,
                       scheduler=scheduler,
                       pet_type=pet_type,
                       lesion=lesion)

In [9]:
trainer.train(30)

Epoch 1/30:   0%|          | 0/823 [00:00<?, ?batch/s]

Epoch 30/30: 100%|██████████| 823/823 [06:57<00:00,  1.97batch/s]


# Evaluation

In [10]:
class ModelTester:
    def __init__(self, path, device, dataloader):
        self.device = device
        self.dataloader = dataloader
        self.load_model(path)
        self.evaluate()

    def load_model(self, path):
        self.model = torch.load(path)
        self.model.to(self.device)

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []
        probabilities = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)
                
                _, predicted = torch.max(outputs, 1)

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())

        return predictions, labels, probabilities

    def calculate_prob_stats(self, probabilities):
        probabilities = np.array(probabilities)
        min_probs = np.min(probabilities)
        max_probs = np.max(probabilities)
        std_probs = np.std(probabilities)
        mean_probs = np.mean(probabilities)

        return min_probs, max_probs, std_probs, mean_probs

    def calculate_percentage(self, value):
        return f'{value*100:.2f}%'

    def evaluate(self):
        predictions, labels, probabilities = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        f1 = f1_score(labels, predictions, average='weighted')

        min_probs, max_probs, std_probs, mean_probs = self.calculate_prob_stats(probabilities)

        print('Evaluation Results:')
        print(f'Confusion Matrix:\n{cm}')
        print(f'Accuracy: {self.calculate_percentage(accuracy)}')
        print(f'F1 Score: {self.calculate_percentage(f1)}')
        print(f'Mean Probability: {self.calculate_percentage(mean_probs)}')
        print(f'Max Probability: {self.calculate_percentage(max_probs)}')
        print(f'Min Probability: {self.calculate_percentage(min_probs)}')
        print(f'Standard Deviation of Probabilities: {std_probs:.4f}')

In [11]:
%%time
base_path = 'eye/Valid'
folder_name = '일반'
'''
['유']
dog: 안검염, 안검종양, 안검내반증, 유루증, 색소침착성각막염, 핵경화, 결막염
cat: 안검염, 결막염, 각막부골편, 비궤양성각막염, 각막궤양
['상', '하']
dog: 궤양성각막질환, 비궤양성각막질환
['초기', '비성숙', '성숙']
dog: 백내장
'''
pet_type = '개'
lesion = '안검종양'
devices = ['스마트폰', '일반카메라']
symptom = ['유']

processor = PathLabelProcessor(base_path=base_path,
                               folder_name=folder_name,
                               pet_type=pet_type,
                               lesion=lesion,
                               devices=devices,
                               symptom=symptom)

labeled_image_paths = processor.labeled_image_paths

eye/Valid/고양이/안구/일반
eye/Valid/개/안구/일반
Total images: 24976, Total JSON files: 24976
Total cases: 13808
Number of symptomatic cases: 49, Number of asymptomatic cases: 13759
CPU times: user 745 ms, sys: 244 ms, total: 989 ms
Wall time: 987 ms


In [15]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240))])

dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

batch_size = 32
num_workers = os.cpu_count()

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

path = '20231211-105003_개_안검종양.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class 1: 13759 samples
  Class 0: 49 samples


100%|██████████| 432/432 [00:20<00:00, 21.25it/s]


Evaluation Results:
Confusion Matrix:
[[    2    47]
 [   18 13741]]
Accuracy: 99.53%
F1 Score: 99.43%
Mean Probability: 50.00%
Max Probability: 100.00%
Min Probability: 0.00%
Standard Deviation of Probabilities: 0.4997
CPU times: user 24.9 s, sys: 6.23 s, total: 31.1 s
Wall time: 28.2 s


In [16]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 0],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...


- Class Counts:
  Class 0: 49 samples


100%|██████████| 2/2 [00:01<00:00,  1.85it/s]

Evaluation Results:
Confusion Matrix:
[[ 2 47]
 [ 0  0]]
Accuracy: 4.08%
F1 Score: 7.84%
Mean Probability: 50.00%
Max Probability: 100.00%
Min Probability: 0.00%
Standard Deviation of Probabilities: 0.4873
CPU times: user 309 ms, sys: 1.49 s, total: 1.8 s
Wall time: 2.22 s


In [17]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 1],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class 1: 13759 samples


100%|██████████| 430/430 [00:20<00:00, 21.43it/s]

Evaluation Results:
Confusion Matrix:
[[    0     0]
 [   18 13741]]
Accuracy: 99.87%
F1 Score: 99.93%
Mean Probability: 50.00%
Max Probability: 100.00%
Min Probability: 0.00%
Standard Deviation of Probabilities: 0.4997
CPU times: user 24.7 s, sys: 6.09 s, total: 30.8 s
Wall time: 27.7 s


In [ ]:
class PreModelTester:
    def __init__(self, path, device, dataloader):
        self.device = device
        self.dataloader = dataloader
        self.model = models.vgg16_bn(pretrained=True)
        self.load_model(path)
        self.evaluate()

    def load_model(self, path):
        self.model = models.vgg16_bn(pretrained=True)
        nr_filters = self.model.classifier[0].in_features
        self.model.classifier = nn.Linear(nr_filters, 1)
        state_dict = torch.load(path, map_location=torch.device("cpu"))
        model_dict = self.model.state_dict()
        state_dict = {k: v for k, v in state_dict.items() if k in model_dict}
        model_dict.update(state_dict)
        self.model.load_state_dict(model_dict)
        self.model = self.model.to(self.device)

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []
        probabilities = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                
                probs = torch.nn.functional.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                probabilities.extend(probs.max(dim=1).values.cpu().numpy())

        return predictions, labels, probabilities

    def calculate_prob_stats(self, probabilities):
        probabilities = np.array(probabilities)
        min_probs = np.min(probabilities)
        max_probs = np.max(probabilities)
        std_probs = np.std(probabilities)
        mean_probs = np.mean(probabilities)

        return min_probs, max_probs, std_probs, mean_probs

    def evaluate(self):
        predictions, labels, probabilities = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        f1 = f1_score(labels, predictions, average='weighted')

        min_probs, max_probs, std_probs, mean_probs = self.calculate_prob_stats(probabilities)

        print("Evaluation Results:")
        print(f"Confusion Matrix:\n{cm}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Mean Probability: {mean_probs:.4f}")
        print(f"Max Probability: {max_probs:.4f}")
        print(f"Min Probability: {min_probs:.4f}")        
        print(f"Standard Deviation of Probabilities: {std_probs:.4f}")

In [ ]:
%%time
dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

path = 'pre_eye.pt'

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)

In [ ]:
%%time
dataset = ImageDataset(
    labeled_image_paths=labeled_image_paths,
    transform=transform
)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)

In [ ]:
%%time
dataset = ImageDataset(
    labeled_image_paths=labeled_image_paths,
    transform=transform
)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)